In [1]:
# Decorators on classes

In [3]:
from fractions import Fraction

In [4]:
# monkey patching, adding methods to an object on runtime

In [6]:
Fraction.speak = lambda self, msg: 'All fractions shall {}'.format(msg)

In [7]:
fr = Fraction(3, 2)

In [8]:
fr.speak("have a denominator")

'All fractions shall have a denominator'

In [9]:
fr2 = Fraction(12, 4)

In [10]:
fr2.speak("have a numerator")

'All fractions shall have a numerator'

In [11]:
fr3 = Fraction(0)

In [12]:
fr3.numerator, fr3.denominator

(0, 1)

In [13]:
# writing directly
Fraction.is_integral = lambda self: self.denominator == 1

In [15]:
fr2.is_integral()

True

In [17]:
fr.is_integral()

False

In [18]:
# monkey patching the fraction

In [19]:
def dec_speak(klass):
    klass.speak = lambda self, msg: "{0} says '{1}'".format(self.__class__.__name__, msg) # you have modified klass, adding a method to it
    return klass

In [20]:
Fraction = dec_speak(Fraction)

In [22]:
f1 = Fraction(2, 13)

In [23]:
f1.speak("please")

"Fraction says 'please'"

In [24]:
class Peterson:
    pass

In [25]:
Peterson = dec_speak(Peterson)

In [26]:
pet = Peterson()

In [27]:
pet.speak("Come over here babe")

"Peterson says 'Come over here babe'"

In [13]:
# debug decorator
from datetime import datetime

def debug_info(obj): # obj can be renamed to self
    info = []
    info.append('time: {0}'.format(datetime.now()))
    info.append('Class; {}'.format(obj.__class__.__name__))
    info.append('id: {}'.format(hex(id(obj))))
    for k, v in vars(obj).items():
        info.append("{0}:{1}".format(k, v))
    return info

In [17]:
def add_debug_info(klass):
    klass.debug = debug_info
    return klass

In [18]:
# We can decorate using Person = add_debug_info(Person)
# OR..

In [19]:
@add_debug_info # this mutates the Person class adding the debug method
class Person:
    def __init__(self, name, birth):
        self.name= name
        self.birth= birth
    def say_hello(self):
        return "{} says hello".format(self.name)

In [20]:
q = Person("Jack Sparrow", 1498)

In [21]:
q.debug()

['time: 2019-05-31 11:20:37.496321',
 'Class; Person',
 'id: 0x7f54042122e8',
 'name:Jack Sparrow',
 'birth:1498']

In [22]:
# decorators must be general enough that they can apply to other classes

In [23]:
p = Person("Tony", 1999)

In [24]:
p.debug()

['time: 2019-05-31 11:24:37.299281',
 'Class; Person',
 'id: 0x7f540427c2e8',
 'name:Tony',
 'birth:1999']

In [31]:
@add_debug_info
class Vehicle:
    def __init__(self, make, model, year, speed):
        self.make = make
        self.model = model
        self.year = year
        self.speed = 0
        
    @property
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self, new_speed):
        self._speed = new_speed  
    

In [32]:
fav = Vehicle('Volk', 'R', 2999, 2000)

In [33]:
fav.debug()

['time: 2019-05-31 11:36:53.342709',
 'Class; Vehicle',
 'id: 0x7f540427c898',
 'make:Volk',
 'model:R',
 'year:2999',
 '_speed:0']

In [34]:
fav.speed = 30

In [35]:
fav.debug()

['time: 2019-05-31 11:37:37.317262',
 'Class; Vehicle',
 'id: 0x7f540427c898',
 'make:Volk',
 'model:R',
 'year:2999',
 '_speed:30']

In [36]:
from math import sqrt

In [37]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    def __repr__(self):
        return "point({0}, {1})".format(self.x, self.y)

In [38]:
p1 = Point(2, 3)

In [39]:
p1

point(2, 3)

In [40]:
abs(p1)

3.605551275463989

In [41]:
p2 = Point(2, 3)

In [43]:
p1 == p2 # python defaults to using memory addresses to handle equality which in this instance returns false

False

In [52]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    def __repr__(self):
        return "point({0}, {1})".format(self.x, self.y)
    def __eq__(self, other):
        if isinstance(other, Point): # other must be an instance of Point
            return self.x == other.x and self.y == other.y
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [53]:
p1, p2, p3 = Point(3, 2), Point(3, 2), Point(2, 3)

In [54]:
p1 == p2

True

In [55]:
p2 == p3

False

In [56]:
p2 < p3

False

In [58]:
p4 = Point(10, 28)

In [59]:
p1 < p4

True

In [69]:
p1 != p3

True

In [64]:
p4 > p2

True

In [89]:
# monkey punching danda fns
# not good python but makes good use of decorators
def complete_ordering(klass):
    if '__eq__' in dir(klass) and '__lt__' in dir(klass):
        klass.__le__ = lambda self, other: self < other or self == other
        klass.__gt__ = lambda self, other: not (self < other) and not (self == other)
        klass.__ge__ = lambda self, other: not (self < other)
    return klass # this line is important if you want to use @ for decorator

In [90]:
@complete_ordering # there is an already defined decorator called total_ordering for this
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    def __repr__(self):
        return "point({0}, {1})".format(self.x, self.y)
    def __eq__(self, other):
        if isinstance(other, Point): # other must be an instance of Point
            return self.x == other.x and self.y == other.y
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)

In [91]:
p1, p2, p3, p4 = Point(3, 2), Point(3, 2), Point(12, 3), (0, 0)

In [92]:
p1 < p2

False

In [93]:
p1 <= p2

True

In [94]:
p1 >= p3

False

In [95]:
p1, p3

(point(3, 2), point(12, 3))

In [97]:
p1 >= p3

False

In [98]:
p1, p2 = Point(1,2), Point(2, 1)

In [99]:
p1 < p2

False

In [100]:
p1 >p2

True

In [101]:
# an already defined decorator for this, just needs one comparison and it generates the rest
from functools import total_ordering

In [124]:
from functools import total_ordering
@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    def __repr__(self):
        return "point({0}, {1})".format(self.x, self.y)
    def __eq__(self, other):
        if isinstance(other, Point): # other must be an instance of Point
            return self.x == other.x and self.y == other.y
    def __gt__(self, other): # this time using gt
        if isinstance(other, Point):
            return abs(self) > abs(other)

In [125]:
p1, p2 = Point(1,2), Point(2, 1)

In [127]:
 p1 > Point(1, 1)

True

In [129]:
p2 <= Point(3, 1)

True

In [117]:
p1 == 3

False

In [123]:
Point(1, 2) == Point(1, 2)

False